In [68]:
!mkdir -p /scratch/sagarsj42/torch-cache
!mkdir -p /scratch/sagarsj42/transformers

import os
os.chdir('/scratch/sagarsj42')
os.environ['TORCH_HOME'] = '/scratch/sagarsj42/torch-cache'
os.environ['TRANSFORMERS_CACHE'] = '/scratch/sagarsj42/transformers'
os.environ['TOKENIZERS_PARALLELISM'] = 'false'

In [64]:
import pandas as pd
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset

from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import Trainer

In [3]:
DATA_PATH = 'IRSE_FIRE_2022_Track_Training_Data_preprocessed.csv'
MODEL_KEY = 'bert-base-uncased'

In [4]:
irse_df = pd.read_csv('IRSE_FIRE_2022_Track_Training_Data_preprocessed.csv')

print(irse_df.info())

irse_df.head(10)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8047 entries, 0 to 8046
Data columns (total 3 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   Comments                  8047 non-null   object
 1   Surrounding Code Context  8047 non-null   object
 2   Class                     8047 non-null   object
dtypes: object(3)
memory usage: 188.7+ KB
None


,Comments,Surrounding Code Context,Class
0,/*test 529*/,-10. int res = 0;\n-9. CURL *curl = NULL;\...,Not Useful
1,/*test 525*/,"-2. fprintf(stderr, ""Usage: lib529 [url] [...",Not Useful
2,/*done*/,"-10. multi_add_handle(m, curl);\n-9. for(;...",Not Useful
3,/*test 529*/,-10. int res = 0;\n-9. CURL *curl = NULL;\...,Not Useful
4,/*test 525*/,"-2. fprintf(stderr, ""Usage: lib529 [url] [...",Not Useful
5,/*argv1 = URL\n * argv2 = proxy\n * argv3 = no...,"-9. *\n-8. * You may opt to use, copy, modif...",Not Useful
6,/*enable ascii/text mode*/,"-10. if(!curl) {\n-9. fprintf(stderr, ""c...",Not Useful
7,/*100 ms*/,"-10. multi_add_handle(mhandle, handle);\n-9....",Not Useful
8,/*on failure don't abort just issue a warning*/,-10. * point the resulting soft limit is ou...,Not Useful
9,/*on failure don't abort just issue a warning*/,-10. * point the resulting soft limit is ou...,Not Useful


In [65]:
tokenizer = AutoTokenizer.from_pretrained(MODEL_KEY)
model = AutoModelForSequenceClassification.from_pretrained(MODEL_KEY)

tokenizer, model

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

(PreTrainedTokenizerFast(name_or_path='bert-base-uncased', vocab_size=30522, model_max_len=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}),
 BertForSequenceClassification(
   (bert): BertModel(
     (embeddings): BertEmbeddings(
       (word_embeddings): Embedding(30522, 768, padding_idx=0)
       (position_embeddings): Embedding(512, 768)
       (token_type_embeddings): Embedding(2, 768)
       (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
       (dropout): Dropout(p=0.1, inplace=False)
     )
     (encoder): BertEncoder(
       (layer): ModuleList(
         (0): BertLayer(
           (attention): BertAttention(
             (self): BertSelfAttention(
               (query): Linear(in_features=768, out_features=768, bias=True)
               (key): Linear(in_features=768, out_features=768, bias=True)
               (val

In [28]:
classes = sorted(irse_df['Class'].value_counts().index.tolist())

classes

['Not Useful', 'Useful']

In [29]:
irse_df['Label'] = irse_df['Class'].apply(lambda l : classes.index(l))

irse_df

,Comments,Surrounding Code Context,Class,Label
0,/*test 529*/,-10. int res = 0;\n-9. CURL *curl = NULL;\...,Not Useful,0
1,/*test 525*/,"-2. fprintf(stderr, ""Usage: lib529 [url] [...",Not Useful,0
2,/*done*/,"-10. multi_add_handle(m, curl);\n-9. for(;...",Not Useful,0
5,/*argv1 = URL\n * argv2 = proxy\n * argv3 = no...,"-9. *\n-8. * You may opt to use, copy, modif...",Not Useful,0
6,/*enable ascii/text mode*/,"-10. if(!curl) {\n-9. fprintf(stderr, ""c...",Not Useful,0
...,...,...,...,...
8042,/*we need an additional DG0-node for a dim-dim...,-10. riaf.back() = true;\n-9. return riaf;...,Useful,1
8043,/*vertex/line/face domination\n(if fe_other is...,-1. return FiniteElementDomination::no_r...,Useful,1
8044,/*there are no requirements between continuous...,-2. if (codim > 0)\n-1. if (dynamic_cast...,Useful,1
8045,/*vertex/line/face domination\n(if fe_other is...,-1. return FiniteElementDomination::no_r...,Useful,1


In [30]:
irse_df = irse_df.drop_duplicates()

irse_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5949 entries, 0 to 8046
Data columns (total 4 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   Comments                  5949 non-null   object
 1   Surrounding Code Context  5949 non-null   object
 2   Class                     5949 non-null   object
 3   Label                     5949 non-null   int64 
dtypes: int64(1), object(3)
memory usage: 232.4+ KB


In [31]:
train_df, dev_df = train_test_split(irse_df, test_size=0.1, 
    stratify=irse_df['Label'])

train_df.shape, dev_df.shape

((5354, 4), (595, 4))

In [72]:
class CommentContextDataset(Dataset):
    def __init__(self, df, tokenizer, max_len=512):
        self.df = df
        self.tokenizer = tokenizer
        self.max_len = 512
        
        self.size = self.df.shape[0]

    def __len__(self):
        return self.size

    def __getitem__(self, idx):
        comment = self.df.iloc[idx]['Comments']
        context = self.df.iloc[idx]['Surrounding Code Context']
        com_con_tok = tokenizer(text=comment, text_pair=context, max_length=self.max_len, 
            padding=True, truncation=True, 
            return_tensors='pt', return_attention_mask=True, return_token_type_ids=True)
        com_con_tok['label'] = self.df['Label'].iloc[idx]

        return com_con_tok


train_ds = CommentContextDataset(train_df, tokenizer)
dev_ds = CommentContextDataset(dev_df, tokenizer)

len(train_ds), len(dev_ds)

(5354, 595)

In [74]:
train_ds[0]

{'input_ids': tensor([[  101,  1013,  1008,  2830,  8170,  2015,  2109,  1999,  2023,  5371,
          1012,  1008,  1013,   102,  1011,  2184,  1012,  1001,  2421,  1000,
          1054, 14277,  3089, 16952,  1012,  1044,  1000,  1011,  1023,  1012,
          1001,  2421,  1000,  1054,  4877, 18223,  1012,  1044,  1000,  1011,
          1022,  1012,  1001,  2421,  1000,  1060,  9067,  4135,  2278,  1012,
          1044,  1000,  1011,  1021,  1012,  1001,  2065, 13629,  2546,  1054,
          2140,  1035,  3075,  1035,  2544,  1011,  1020,  1012,  1001,  9375,
          1054,  2140,  1035,  3075,  1035,  2544,  1000,  1019,  1012,  1015,
          1000,  1011,  1019,  1012,  1001,  2203, 10128,  1011,  1018,  1012,
          1001,  2065, 13629,  2546,  1054,  2140,  1035,  3191,  4179,  1035,
          2544,  1011,  1017,  1012,  1001,  9375,  1054,  2140,  1035,  3191,
          4179,  1035,  2544,  1014,  2595,  2692, 12376,  2487,  1011,  1016,
          1012,  1001,  2203, 10128,  

In [75]:
trainer = Trainer(
    model=model,
    train_dataset=train_ds, eval_dataset=dev_ds
)

trainer

No `TrainingArguments` passed, using `output_dir=tmp_trainer`.
PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [76]:
trainer.train()

/home2/sagarsj42/miniconda3/lib/python3.9/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 5354
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 2010
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"


RuntimeError: stack expects each tensor to be equal size, but got [1, 57] at entry 0 and [1, 54] at entry 1